---

# CSCI 3202, Fall 2023
# Mancala Project - Outline
# Caleb Lehman


In [2]:
import copy, math, time, random
random.seed(109)

In [3]:
class MancalaAI:
    def __init__(self, depth, state):
        
        self.depth = depth
        self.state = state
    
    def valid_moves(self,state,player):

        moves = []
        if player == 1:

            for i in range(state.pits_per_player):
                if state.board[i] > 0:
                    moves.append(i+1)
        
        if player == 2:

            for i in range(state.pits_per_player):
                if state.board[i+state.pits_per_player + 1] > 0:
                    moves.append(i+1)

        return moves


    def minimax(self, state, depth, maximizing_player, cur_player):
        
        # stop condition, game is over or depth has been reached

        if depth == 0 or state.winning_eval() == True:
            return self.evaluate_state(state)
        
        
        if maximizing_player:
            # generate all possible states for the maximizing player, and recurse 
            # until you reach the stop condition - terminal state

            value = -100000
            possible_valid_moves = self.valid_moves(state,1)
            
            for i in range(len(possible_valid_moves)):
                
                game = copy.deepcopy(state)
                game.play(possible_valid_moves[i],1)

                value = max(value,self.minimax(game, depth-1, False,2)) # minimizing players' move
        else:
            # generate all possible states for the maximizing player, and recurse
            
            value = 100000
            possible_valid_moves = self.valid_moves(state,2)
            
            for i in range(len(possible_valid_moves)):
                
                game = copy.deepcopy(state)
                game.play(possible_valid_moves[i],2)

                value = min(value,self.minimax(game, depth-1, True, 1)) # maximizing players' move
        
        return value
    
    def minimax_alpha_beta(self, state, depth, alpha, beta, maximizing_player, cur_player):
        
        if depth == 0 or state.winning_eval() == True:
            return self.evaluate_state(state)
        

        if maximizing_player:

            value = -100000
            possible_valid_moves = self.valid_moves(state,1)

            for i in range(len(possible_valid_moves)):

                game = copy.deepcopy(state)
                game.play(possible_valid_moves[i],1)

                value = max(value,self.minimax_alpha_beta(game,depth -1,alpha,beta,False,2))

                if value >= beta:
                    
                    return value
                    
                
                alpha = max(alpha,value)
        
        else:
            value = 100000
            possible_valid_moves = self.valid_moves(state,2)

            for i in range(len(possible_valid_moves)):

                game = copy.deepcopy(state)
                game.play(possible_valid_moves[i],2)

                value = min(value,self.minimax_alpha_beta(game,depth-1,alpha,beta,True,1))

                if value <= alpha:
                    
                    return value
                    
                    
                beta = min(beta,value)
        
        return value



    def best_move(self, state,cur_player, alpha_beta = False):
        
        if cur_player == 1:
            
            other_player = 2
        
        else:
            
            other_player = 1


        if alpha_beta == False:

            possible_valid_moves = self.valid_moves(state,cur_player)

            best_move = possible_valid_moves[0]

            value = -1000000

            for i in range(len(possible_valid_moves)):

                game = copy.deepcopy(state)

                game.play(possible_valid_moves[i],cur_player)

                if cur_player == 2:
                    maximizing_player = True
                else:
                    maximizing_player = False

                val = self.minimax(game,self.depth,maximizing_player,other_player)

                if val > value:
                    value = val
                    best_move = possible_valid_moves[i]

            return best_move

        else:

            possible_valid_moves = self.valid_moves(state,cur_player)

            best_move = possible_valid_moves[0]

            value = -100000

            alpha = -10000000
            beta  =  10000000

            for i in range(len(possible_valid_moves)):

                game = copy.deepcopy(state) 

                game.play(possible_valid_moves[i],cur_player)

                if cur_player == 2:
                    maximizing_player = True
                else:
                    maximizing_player = False
                
                val = self.minimax_alpha_beta(game,self.depth,alpha,beta,maximizing_player,other_player)

                if val > value:
                    value = val
                    best_move = possible_valid_moves[i]
            
            return best_move

        
        
        
        

    def evaluate_state(self, state):
        # Utility function  :- Difference between P1 mancala and p2 mancala
        return state.board[state.p1_mancala_index] - state.board[state.p2_mancala_index]
        
        

In [4]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        self.pits_per_player = pits_per_player
        self.stones_per_pit = stones_per_pit
        self.board = [stones_per_pit] * ((pits_per_player + 1) * 2)
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player + 1, len(self.board) - 2]
        self.p2_mancala_index = len(self.board) - 1
    
        self.num_plays = 0
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0
        self.p1_wins = 0
        self.p2_wins = 0
        self.ties = 0
        

    def display_board(self):
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1] + 1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1] + 1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i + 1, player_1_pits[i], player_2_pits[-(i + 1)],
                                                       self.pits_per_player - i))
            else:
                print('{} -> | {} | {} | <- {}'.format(i + 1, player_1_pits[i], player_2_pits[-(i + 1)],
                                                      self.pits_per_player - i))

        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
    
    def random_move_generator(self,cur_player):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """

        if self.winning_eval() == True:
            return -1
        
        val = random.randint(1,self.pits_per_player)

        while self.valid_move(val,cur_player) == False:
            val = random.randint(1,self.pits_per_player)
        
        return val
        
        
    def generate_board(self):
        self.board = [self.stones_per_pit] * ((self.pits_per_player+1) * 2)
        self.current_player = 1
        self.moves = []
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def valid_move(self, pit,cur_player):

        if cur_player == 1:
            if pit > self.pits_per_player or self.board[pit-1] == 0:
                return False
            else:
                return True
        else:
            if pit > self.pits_per_player or self.board[pit + self.pits_per_player] == 0:
                return False
            else:
                return True        
      
       
        

    def play(self, pit, cur_player, debug = False):
        
        if pit == -1:
            return True
        
        if self.valid_move(pit,cur_player) == False:
            print("invalid move")
            return True
        
        if cur_player == 1:
                
            stones = self.board[pit-1]
            
            index = 0

            for i in range(stones):
                
                self.board[pit - 1] = self.board[pit - 1] - 1

                index = ( (pit + i) % len(self.board))

                if index == self.p2_mancala_index:
                        stones = stones + 1
                        
                else:
                    self.board[index] = self.board[index] + 1
                    
            if index < self.p1_mancala_index and self.board[index] == 1 and self.board[self.p1_mancala_index + (self.p1_mancala_index - index)] != 0:
                opp = self.p1_mancala_index + (self.p1_mancala_index - index)

                self.board[self.p1_mancala_index] = self.board[self.p1_mancala_index] + 1 + self.board[opp]

                self.board[index] = 0
                self.board[opp] = 0
                    
                
                self.moves.append((1,pit))
                self.current_player = 2

                
        if cur_player == 2:

                start = pit + self.pits_per_player
                stones = self.board[start]

                index = 0
                for i in range(stones):
                    self.board[start]  = self.board[start] -1

                    index = (start + i + 1) % (len(self.board))
                        

                    if index == self.p2_mancala_index:
                        self.board[self.p2_mancala_index] = self.board[self.p2_mancala_index] + 1

                    elif index == self.p1_mancala_index:
                        stones = stones + 1
                    else:
                        self.board[index] = self.board[index] + 1

                if index != self.p2_mancala_index and index > self.p1_mancala_index and self.board[index] == 1 and self.board[self.p1_mancala_index - (index - self.p1_mancala_index)] != 0:
                    opp = self.p1_mancala_index - (index - self.p1_mancala_index)

                    self.board[self.p2_mancala_index] = self.board[self.p2_mancala_index] + 1 + self.board[opp]

                    self.board[index] = 0
                    self.board[opp] = 0
                    
                
                self.moves.append((2,pit))
                self.current_player = 1

            
            
        self.num_plays = self.num_plays + 1

        return self.winning_eval()



    def winning_eval(self):
        
        game_over1 = True
        game_over2 = True

        for i in range(self.pits_per_player):
            if self.board[i] > 0:
                game_over1 = False
        
        for i in range(self.pits_per_player + 1, len(self.board)-1):
            if self.board[i] > 0:
                game_over2 = False
        
        if game_over1 or game_over2:

            return True
        
        return False
    
    def points(self):
        
        if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                self.p1_wins = self.p1_wins + 1
            
        if self.board[self.p2_mancala_index] > self.board[self.p1_mancala_index]:
                self.p2_wins = self.p2_wins +1
            
        if self.board[self.p1_mancala_index] == self.board[self.p2_mancala_index]:
                self.ties = self.ties + 1
    
    def rand_match_analysis(self, games = 100):
        
        for i in range(games):

            move = False
            
            while move == False:
                
                
                move = self.play(self.random_move_generator(1),1,False)
               

                self.num_plays = self.num_plays + 1
                if move == True:
                    break

                move = self.play(self.random_move_generator(2),2,False)
                self.num_plays = self.num_plays + 1
            
            self.points()

            self.generate_board()
    
    
    def AI_vs_rand(self,depth,games):
        
        for i in range(games):

            move = False

            while move == False:

                AI = MancalaAI(depth,self)

                game = copy.deepcopy(self)

                AI_move = AI.best_move(game,1,False)


                move = self.play(AI_move,1,False)

                self.num_plays = self.num_plays+1

                if move == True:
                    break

                move = self.play(self.random_move_generator(2),2,False)
                self.num_plays = self.num_plays + 1
            
            self.points()

            self.generate_board()
    
    def AI_alpha_beta_vs_rand(self,depth,games):

        for i in range(games):

            move = False

            while move == False:

                AI = MancalaAI(depth,self)

                game = copy.deepcopy(self)

                AI_move = AI.best_move(game,1,True)

                move = self.play(AI_move,1,False)

                self.num_plays = self.num_plays + 1

                if move == True:
                    break

                move = self.play(self.random_move_generator(2),2,False)
                self.num_plays = self.num_plays + 1
            
            self.points()

            self.generate_board()


                

       


In [5]:

##1. Run 100 games of random vs random

rand_game = Mancala()

num_games = 100
rand_game.rand_match_analysis(num_games)

print("P1 wins: " + str(rand_game.p1_wins))
print("P2 wins: " + str(rand_game.p2_wins))
print("Ties: "+ str(rand_game.ties))
print("Avg Plays: "+ str(rand_game.num_plays/num_games))

# On the first time compiling, we see 49 wins for player one, 43 for player two, and 8 ties, with an average game lasting around 81 plays
# So the win percenage is around 50% for any of the players, which makes sense as both are playing randomly





P1 wins: 49
P2 wins: 43
Ties: 8
Avg Plays: 80.84


So, after playing 100 games of randomly chosen moves from both players, we can see that the winrate for p1 hovers around 50%. Depending on the seed or how many times it was compiled, p2 could win more often. So, we can argue that the winrate is around 50% when placing two random opponants against eachother.

This is expected as every move is chosen randomly; however, you could argue that player 1 might have a higher than 50% winrate, simply due to the fact that the first person has first move advantage. Looking at a sample of 10,000 games, p1 did win more often than p2, almost confirming a first move advantage in this game. So on a smaller sample size of 100 games, it would be hard to see, but on a larger sample size, we can argue that player 1 will win more often than not, even if both players choose pits randomlly.

Each game takes around 80 to 90 moves to complete, meaning each player gets around 40 to 45 turns.


In [6]:
## AI versus random player, 100 games, depth of 5 piles, no alpha beta

gameAI = Mancala()

gameAI.AI_vs_rand(5,100)

print("AI wins: " + str(gameAI.p1_wins))
print("Random player wins: " + str(gameAI.p2_wins))
print("Ties: " + str(gameAI.ties))
print("Avg number of plays:" + str(gameAI.num_plays/100))

## On first compilation with a depth of five piles, the AI bot will always win every game. To play the 100 games, it usually takes anywhere between 4 to 4 and a half minutes.
## on average, there are about 60 to 65 turns


AI wins: 100
Random player wins: 0
Ties: 0
Avg number of plays:63.1


Above are the results for the minimax algorithm with a depth of 5 piles against a random player. The AI will win 100% of the games. Even at a depth of 2 piles, it will win every game. 

This algorithm usually takes anywhere from 4 to 4 and a half minutes to play 100 games,  and has an average number of moves around 60 to 66 moves, so each player got around 30 to 33 turns.

I would say that the AI player is definitively better than random chance. Even looking at the AI with 0 piles, it still wins over 90% of the time. Once you add the ability for the AI to look deeper into the game, it will never lose. The goal of the AI is to maximize the difference between the stones in each players mancala, which is how you win the game. It will always be able to do this more optimally than the random opponent. You could argue that if the random opponent played some exact move order, it could beat the AI; however, given a sample size of finite games, the AI will always perform better than random chance.

In [7]:
## AI using alpha beta minmax vs random player, 100 games, depth of 5 piles

gameAI = Mancala()

gameAI.AI_alpha_beta_vs_rand(5,100)

print("AI wins: " + str(gameAI.p1_wins))
print("Random player wins: " + str(gameAI.p2_wins))
print("Ties: " + str(gameAI.ties))
print("Avg number of plays:" + str(gameAI.num_plays/100))



AI wins: 100
Random player wins: 0
Ties: 0
Avg number of plays:61.22


Here we are again using the minimax algorithm with a depth of 5 piles, but also including the alpha beta algorithm to prune non-optimal branches. The AI still won 100% of the games.

However, looking at the run time, we can see that it takes anywhere from 1 to 1 and half minutes, meaning the alpha beta version of minimax runs 70% faster, while still winning every game against the random opponent. Each game took around .6 to .9 seconds to complete.

Here, the amount of moves to win is still around the same, with each player getting around 30 to 33 turns.

The minimax alpha beta function will do the same thing as the normal minimax function, just in a faster time. The alpha beta allows non optimal branches to be pruned which saves time, but since those pruned branches had no optimal moves, the normal minimax function would not have picked them anyways. So, ideally given the same board, both minimax and alpha beta minimax will pick the same optimal move and should win in the same number of turns, but alpha beta minimax will do it in a much quicker time.

In [1]:
## AI using alpha beta with depth of 10 piles


gameAI = Mancala()

gameAI.AI_alpha_beta_vs_rand(10,100)

print("AI wins: " + str(gameAI.p1_wins))
print("Random player wins: " + str(gameAI.p2_wins))
print("Ties" + str(gameAI.ties))
print("Avg number of plays:" + str(gameAI.num_plays/100))



The 100 games at 10 piles is currently running while I am typing this, and will probably be running for long after I finish. I tried the alpha beta minimax algorithm with a depth of 10 piles for one game, and it took around 1 and a half minutes to finish. So, 100 games should take over 2 hours to run. 

My educated guess is that this AI will win 100% of the games, as even with only 1 - 2 piles, the AI won every game. Increasing the number of piles will most likely cut down slightly on move count, but in my opinion, is not worth the cost and time it takes to run.

If the AI is already winning 100 percent of the games and the goal is only to win, there is no reason to keep increasing the pile count. If we wanted to optimize play and move count, we can see that increasing the number of piles exponentially increases the run time. If the goal was to optimize play while also winning, it might be better to keep a middle ground depth, while changing our utility function to increase play optimization.


The 100 games at depth 10 piles took 168 minutes to run. It had an average of 68.5 moves, which is similar to the games with lower pile counts. So, in fact, increasing the number of piles had little to no impact on the move count. From this, it would be safe to say that increasing the depth of piles is not the best way to increase efficiency of play. A far better choice would be to adjust the utility function, or modify the decision making algorithm.

